I'm gonna overwrite a lot of this notebook's old content. I changed the way I'm calculating wt, and wanna test that my training worked. 

In [1]:
from pearce.emulator import NashvilleHot
import numpy as np
from os import path

In [2]:
from GPy.kern import *

In [3]:
import matplotlib
#matplotlib.use('Agg')
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set()

In [4]:
training_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_v3/PearceXiggCosmo.hdf5'
test_file = '/home/users/swmclau2/scratch/xi_gg_zheng07_cosmo_test_v3/PearceXiggCosmoTest.hdf5'

In [5]:
fixed_params = {'z': 0.0}#, 'r':0.0958} #TODO fixed r broken

In [6]:
hyperparams = {'kernel': ( Matern32(input_dim=7, ARD=False)+ RBF(input_dim=7, ARD=False) + Bias(input_dim=7),
                           Matern32(input_dim=4, ARD=False)+ RBF(input_dim=4, ARD=False) + Bias(input_dim=4)), \
               'optimize': True}

In [7]:
emu = NashvilleHot(training_file, fixed_params = fixed_params, hyperparams=hyperparams\
                  , downsample_factor = 0.1)

 /home/users/swmclau2/.local/lib/python2.7/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in divide
 /home/users/swmclau2/.local/lib/python2.7/site-packages/GPy/kern/src/stationary.py:454: RuntimeWarning:overflow encountered in multiply
 /home/users/swmclau2/.local/lib/python2.7/site-packages/GPy/kern/src/rbf.py:51: RuntimeWarning:overflow encountered in square
 /home/users/swmclau2/.local/lib/python2.7/site-packages/GPy/kern/src/stationary.py:457: RuntimeWarning:overflow encountered in multiply


In [9]:
emu._save_as_default_kernel()

In [10]:
x1, x2, y, yerr, cov, info = emu.get_data(test_file, fixed_params)

In [11]:
hod_idx, cosmo_idx = 42,  17

In [12]:
params = dict(zip(emu.get_param_names(), np.hstack([x1[cosmo_idx], x2[hod_idx]])))
y_pred1 = emu.emulate_wrt_r(params)

In [13]:
from pearce.emulator import DEFAULT_METRIC_NH_PICKLE_FNAME
print DEFAULT_METRIC_NH_PICKLE_FNAME

/home/users/swmclau2/.local/lib/python2.7/site-packages/pearce/emulator/default_nh_metrics.pkl


In [14]:
emu2 = NashvilleHot(training_file, fixed_params=fixed_params, downsample_factor = 0.1)

In [15]:
y_pred2 = emu2.emulate_wrt_r(params)